In [1]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import PIL.Image
import imageio
import tfutil
import matplotlib.pyplot as plt
import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Replicate the training set

In [22]:
# Read the labels
labels = np.load('//nas5.pmi.rwth-aachen.de/mri-scratch/DeepLearning/CheXpert_Dataset/labels/label_CheXpert.npy'
                , allow_pickle=True)
vals = labels.item().values()
values = np.array(list(vals))
labels = values
labels[:20]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0

In [20]:
# Initialize TensorFlow session.
tf.InteractiveSession()

# Import official CelebA-HQ networks.
with open('network-snapshot-012000.pkl', 'rb') as file:
    G, D, Gs = pickle.load(file)

W1204 10:55:10.065437 10724 deprecation_wrapper.py:119] From C:\Users\peter\2080Ti\Jupyter\CheXpert_256\tfutil.py:468: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1204 10:55:10.067432 10724 deprecation_wrapper.py:119] From C:\Users\peter\2080Ti\Jupyter\CheXpert_256\tfutil.py:471: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1204 10:55:10.067432 10724 deprecation_wrapper.py:119] From C:\Users\peter\2080Ti\Jupyter\CheXpert_256\tfutil.py:471: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W1204 10:55:10.068429 10724 deprecation_wrapper.py:119] From C:\Users\peter\2080Ti\Jupyter\CheXpert_256\tfutil.py:472: The name tf.get_variable_scope is deprecated. Please use tf.compat.v1.get_variable_scope instead.

W1204 10:55:10.068429 10724 deprecation_wrapper.py:119] From C:\Users\peter\2080Ti\Jupyter\CheXpert_256\tfutil.py:475: The name tf.placeholder is deprecat

In [23]:
batch = 1
used_labels = []
used_imgname = []
latents_raw = np.random.RandomState(1000).randn(labels.shape[0], *Gs.input_shapes[0][1:])
total_num = latents_raw.shape[0]
for n in range (int(total_num / batch)):
    latent_vec = latents_raw[n * batch: (n+1) * batch, :]
    label_vec = labels[n * batch: (n+1) * batch, :]
    used_labels.append(label_vec)
    images = Gs.run(latent_vec, label_vec)
    images = np.clip(np.rint((images + 1.0) / 2.0 * 255.0), 0.0, 255.0).astype(np.uint8) # [-1,1] => [0,255]
    images = images.transpose(0, 2, 3, 1) # NCHW => NHWC
    save_images = np.squeeze(images, axis = -1)
    
    data_dir = "//nas5.pmi.rwth-aachen.de/mri-scratch/DeepLearning/Fake_Stanford/"
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    for idx in range(save_images.shape[0]):
        image_idx = idx + batch * n
        store_name = 'fake_%d.png' %image_idx
        used_imgname.append(store_name)
        store_path = os.path.join(data_dir, store_name)
        imageio.imwrite(store_path, save_images[idx])

print(len(used_labels))
print(len(used_imgname))

191229
191229


In [25]:
labels[:20]
used_labels[:20]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0

[array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
 array([[0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0.]]),
 array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]),
 array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]]),
 array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1.]]),
 array([[0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.]]),
 array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.]]),
 array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.]]),
 array([[0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]),
 array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,